In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 850M (CNMeM is disabled, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
path = "data/sign_mix_expedition3/"

In [3]:
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *

Using Theano backend.


In [4]:
batch_size=32

# Defining the model and doing a transfer of weights from vgg16

In [5]:
# This is for extracting out the convolutional part of vgg16
vgg = Vgg16()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [6]:
for layer in conv_layers:
    layer.trainable = False

In [7]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(24, activation='softmax')
        ]
p=0.8
bn_fc_layers = get_bn_da_layers(p)

In [8]:
hand_sign_model = Sequential(conv_layers + bn_fc_layers)
hand_sign_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Generate batches for the data

In [9]:
# Generate batches for all our data
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(path+'test', batch_size=batch_size, shuffle=False)

Found 1748 images belonging to 24 classes.
Found 336 images belonging to 24 classes.
Found 1 images belonging to 1 classes.


In [10]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 1748 images belonging to 24 classes.
Found 336 images belonging to 24 classes.
Found 1 images belonging to 1 classes.


# Get augmented training batches
Do adjust data multiplication factor below

In [11]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.15, 
                                 channel_shift_range=20, width_shift_range=0.15)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=True)

Found 1748 images belonging to 24 classes.


In [12]:
da_trn_labels = np.concatenate([trn_labels]*4)

In [13]:
da_batches.nb_sample

1748

# Lets try fitting to this

In [23]:
hand_sign_model.fit_generator(da_batches, samples_per_epoch=da_batches.nb_sample*4, nb_epoch=10,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/10
6992/6992 [==============================] - 354s - loss: 0.6637 - acc: 0.7850 - val_loss: 0.2161 - val_acc: 0.9286
Epoch 2/10
6992/6992 [==============================] - 353s - loss: 0.6417 - acc: 0.7925 - val_loss: 0.2205 - val_acc: 0.9256
Epoch 3/10
6992/6992 [==============================] - 371s - loss: 0.6432 - acc: 0.7885 - val_loss: 0.2327 - val_acc: 0.9196
Epoch 4/10
6992/6992 [==============================] - 378s - loss: 0.6057 - acc: 0.8026 - val_loss: 0.2227 - val_acc: 0.9256
Epoch 5/10
6992/6992 [==============================] - 370s - loss: 0.6159 - acc: 0.8009 - val_loss: 0.2321 - val_acc: 0.9196
Epoch 6/10
6992/6992 [==============================] - 374s - loss: 0.5927 - acc: 0.8053 - val_loss: 0.2381 - val_acc: 0.9196
Epoch 7/10
6992/6992 [==============================] - 370s - loss: 0.6032 - acc: 0.8039 - val_loss: 0.2346 - val_acc: 0.9167
Epoch 8/10
6992/6992 [==============================] - 375s - loss: 0.5959 - acc: 0.8079 - val_loss: 0.2480 - 

In [24]:
hand_sign_model.save_weights(path+'models/da_conv8_1_at80.h5')

In [14]:
# Do only when weights have been saved already
hand_sign_model.load_weights(path+'models/da_conv8_1_at71.h5')

# Lets test test test

In [19]:
final_labels = {}
for x,y in batches.class_indices.items():
    final_labels[y]=x

In [20]:
test_batches = get_batches(path+'test', batch_size=batch_size, shuffle=False)

Found 1 images belonging to 1 classes.


In [21]:
preds = hand_sign_model.predict_generator(test_batches, test_batches.nb_sample)

In [22]:
for i, x in enumerate(preds[0]):
    print(final_labels[i], x)

a 0.183358
b 0.00023903
c 0.000481377
d 0.000301477
e 0.0616198
f 0.000167032
g 0.000389257
h 0.000382179
i 0.00173031
k 5.98134e-05
l 1.44814e-05
m 0.00887039
n 0.01836
o 0.003272
p 0.00112842
q 0.000388247
r 0.000142838
s 0.53325
t 0.174555
u 0.000123415
v 8.04657e-06
w 1.33755e-05
x 0.00819007
y 0.00295565


In [33]:
hand_sign_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=1,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
hand_sign_model.save_weights(path+'models/da_conv8_1.h5')

Epoch 1/1
62481/62481 [==============================] - 4478s - loss: 0.8278 - acc: 0.7240 - val_loss: 0.2163 - val_acc: 0.9428


In [61]:
final_labels = {}
for x,y in batches.class_indices.items():
    final_labels[y]=x
final_labels

{0: 'a',
 1: 'b',
 2: 'c',
 3: 'd',
 4: 'e',
 5: 'f',
 6: 'g',
 7: 'h',
 8: 'i',
 9: 'k',
 10: 'l',
 11: 'm',
 12: 'n',
 13: 'o',
 14: 'p',
 15: 'q',
 16: 'r',
 17: 's',
 18: 't',
 19: 'u',
 20: 'v',
 21: 'w',
 22: 'x',
 23: 'y'}